In [2]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return 'NVIDIA GeForce RTX 3050'

True
NVIDIA GeForce RTX 3050


In [3]:
import pandas as pd

# Load the Excel file
oasis_data = pd.read_excel('D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\oasis_cross-sectional.xlsx')

# Display the first few rows
print(oasis_data.head())

              ID M/F Hand  Age  Educ  SES  MMSE  CDR  eTIV   nWBV    ASF  \
0  OAS1_0001_MR1   F    R   74   2.0  3.0  29.0  0.0  1344  0.743  1.306   
1  OAS1_0002_MR1   F    R   55   4.0  1.0  29.0  0.0  1147  0.810  1.531   
2  OAS1_0003_MR1   F    R   73   4.0  3.0  27.0  0.5  1454  0.708  1.207   
3  OAS1_0004_MR1   M    R   28   NaN  NaN   NaN  NaN  1588  0.803  1.105   
4  OAS1_0005_MR1   M    R   18   NaN  NaN   NaN  NaN  1737  0.848  1.010   

   Delay  
0    NaN  
1    NaN  
2    NaN  
3    NaN  
4    NaN  


In [4]:
import os

# Define the path to your image dataset
image_dir = 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets'

# Create a mapping from ID to image paths
id_to_images = {}
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.png'):
            id_part = file.split('_')[0] + '_' + file.split('_')[1] + '_' + file.split('_')[2]
            if id_part not in id_to_images:
                id_to_images[id_part] = []
            id_to_images[id_part].append(os.path.join(root, file))

# Display an example
print(id_to_images['OAS1_0001_MR1'])

['D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_121.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_122.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_123.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_124.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_125.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_126.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_127.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_128.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1.nii_slice_129.png', 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\NonDemented\\OAS1_0001_MR1_1

In [5]:
# Add a column for image paths
oasis_data['ImagePaths'] = oasis_data['ID'].map(id_to_images)

# Drop rows with no corresponding images
oasis_data = oasis_data.dropna(subset=['ImagePaths'])

# Display the updated dataframe
print(oasis_data.head())

              ID M/F Hand  Age  Educ  SES  MMSE  CDR  eTIV   nWBV    ASF  \
0  OAS1_0001_MR1   F    R   74   2.0  3.0  29.0  0.0  1344  0.743  1.306   
1  OAS1_0002_MR1   F    R   55   4.0  1.0  29.0  0.0  1147  0.810  1.531   
2  OAS1_0003_MR1   F    R   73   4.0  3.0  27.0  0.5  1454  0.708  1.207   
3  OAS1_0004_MR1   M    R   28   NaN  NaN   NaN  NaN  1588  0.803  1.105   
4  OAS1_0005_MR1   M    R   18   NaN  NaN   NaN  NaN  1737  0.848  1.010   

   Delay                                         ImagePaths  
0    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  
1    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  
2    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  
3    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  
4    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  


In [6]:
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F

# Define a fixed sequence length
MAX_SEQ_LENGTH = 10  # Adjust based on your dataset

class DementiaDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_paths = row['ImagePaths']
        
        # Load all images for the given ID
        images = []
        for path in image_paths:
            image = Image.open(path).convert('RGB')  # Convert to RGB
            print(image)
            if self.transform:
                image = self.transform(image)  # Apply transformations (including resizing)
            images.append(image)
        
        # Pad or truncate the sequence to a fixed length
        if len(images) > MAX_SEQ_LENGTH:
            images = images[:MAX_SEQ_LENGTH]  # Truncate
        elif len(images) < MAX_SEQ_LENGTH:
            # Pad with zero tensors
            padding = [torch.zeros_like(images[0]) for _ in range(MAX_SEQ_LENGTH - len(images))]
            images.extend(padding)
        
        # Stack images into a single tensor
        images = torch.stack(images)  # Now all sequences have the same length
        
        # Extract demographic data
        demographics = torch.tensor([
            row['Age'],
            row['Educ'],
            1 if row['M/F'] == 'M' else 0,  # Encode gender as 0 or 1
            1 if row['Hand'] == 'R' else 0   # Encode handedness as 0 or 1
        ], dtype=torch.float32)
        
        # Extract label (encode the dementia categories as integers)
        label_map = {'NonDemented': 0, 'VeryMildDemented': 1, 'MildDemented': 2, 'ModerateDemented': 3}
        label = label_map[row['DementiaCategory']]
        
        return images, demographics, label



In [7]:
import torch.nn as nn
import torch.nn.functional as F

class DementiaModel(nn.Module):
    def __init__(self):
        super(DementiaModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc1 = nn.Linear(64 * 58 * 116 + 4, 128)  # Add 4 for demographic features
        self.fc2 = nn.Linear(128, 4)  # Output layer

    def forward(self, images, demographics):
        batch_size = images.size(0)
        num_images = images.size(1)
        
        # Process each image through the CNN
        cnn_features = []
        for i in range(num_images):
            features = self.cnn(images[:, i])
            #print(f"CNN output shape: {features.shape}")  # Should be [8, 64, 58, 116]
            features = features.view(batch_size, -1)  # Flatten the features
            #print(f"Flattened features shape: {features.shape}")  # Should be [8, 430592]
            cnn_features.append(features)
        
        # Average features across all images
        cnn_features = torch.mean(torch.stack(cnn_features), dim=0)
        
        # Concatenate with demographic data
        combined = torch.cat((cnn_features, demographics), dim=1)
        #print(f"Combined shape: {combined.shape}")  # Should be [8, 430596]
        
        # Final fully connected layer
        output = self.fc2(F.relu(self.fc1(combined)))
        return output

In [8]:
import os

# Define the path to your image dataset
image_dir = 'D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets'

# Create a mapping from image paths to their dementia category
image_to_category = {}
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.png'):
            # Extract the subfolder name (dementia category)
            category = os.path.basename(root)
            image_to_category[os.path.join(root, file)] = category

# Add the dementia category to the DataFrame
oasis_data['DementiaCategory'] = oasis_data['ImagePaths'].apply(
    lambda paths: image_to_category.get(paths[0], None)  # Use the first image's category for the ID
)

# Drop rows with no dementia category
oasis_data = oasis_data.dropna(subset=['DementiaCategory'])

# Display the updated dataframe
print(oasis_data.head())

              ID M/F Hand  Age  Educ  SES  MMSE  CDR  eTIV   nWBV    ASF  \
0  OAS1_0001_MR1   F    R   74   2.0  3.0  29.0  0.0  1344  0.743  1.306   
1  OAS1_0002_MR1   F    R   55   4.0  1.0  29.0  0.0  1147  0.810  1.531   
2  OAS1_0003_MR1   F    R   73   4.0  3.0  27.0  0.5  1454  0.708  1.207   
3  OAS1_0004_MR1   M    R   28   NaN  NaN   NaN  NaN  1588  0.803  1.105   
4  OAS1_0005_MR1   M    R   18   NaN  NaN   NaN  NaN  1737  0.848  1.010   

   Delay                                         ImagePaths  DementiaCategory  
0    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...       NonDemented  
1    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...       NonDemented  
2    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...  VeryMildDemented  
3    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...       NonDemented  
4    NaN  [D:\Projetos\AlzheimerEarlyDetection_v3\datase...       NonDemented  


In [9]:
from torch.utils.data import DataLoader

torch.cuda.empty_cache()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((116, 232)),  # Resize to match your image size
    transforms.ToTensor(),          # Convert to tensor
])

# Create the dataset
dataset = DementiaDataset(oasis_data, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=8, pin_memory=True)

In [ ]:
#from torch.amp import GradScaler, autocast

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')  # Deve imprimir 'cuda'

model = DementiaModel().to(device) 
print(next(model.parameters()).device) # Deve imprimir 'cuda:0'

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scaler = GradScaler()

# for epoch in range(10):  # Number of epochs
#     for images, demographics, labels in dataloader:
#         images, demographics, labels = images.to(device), demographics.to(device), labels.to(device)
        
#         optimizer.zero_grad()
#         with autocast():  # Enable mixed precision
#             outputs = model(images, demographics)
#             loss = criterion(outputs, labels)
            
#         scaler.scale(loss).backward()  # Scale loss and backpropagate
#         scaler.step(optimizer)  # Update weights
#         scaler.update()  # Update the scaler
    
#     print(f'Epoch {epoch+1}, Loss: {loss.item()}')

print('antes do for')
for epoch in range(10):  # Número de épocas
    epoch_loss = 0  
    print('entrou no for 1')
    for images, demographics, labels in dataloader:
        print(images)
        print(demographics)
        print(labels)
        print('antes do images, demographics, labels = images.to(device), demographics.to(device), labels.to(device)')
        images, demographics, labels = images.to(device), demographics.to(device), labels.to(device)
        print('depois do images, demographics, labels = images.to(device), demographics.to(device), labels.to(device)')

        optimizer.zero_grad()
        outputs = model(images, demographics)  # Forward Pass
        loss = criterion(outputs, labels)  # Compute Loss

        loss.backward()  # Backpropagation
        optimizer.step()  # Update Weights

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}')

Using device: cuda
cuda:0
antes do for
entrou no for 1


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Example: Plot age vs. predicted dementia severity
predictions = []
ages = []
with torch.no_grad():
    for images, demographics, labels in dataloader:
        images, demographics = images.to(device), demographics.to(device)
        outputs = model(images, demographics)
        predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        ages.extend(demographics[:, 0].cpu().numpy())

plt.scatter(ages, predictions)
plt.xlabel('Age')
plt.ylabel('Predicted Dementia Severity')
plt.show()